# **Parser**

In [2]:
# <include>
from urllib.request import urlopen
import pandas as pd  
from pprint import pprint
import json

# your main file is insanely large, and made the runtime crash
lol = "https://cdn.merakianalytics.com/riot/lol/resources/latest/en-US/champions.json"
# using a reduce version instead, with just the first two players
lol_reduced = "https://raw.githubusercontent.com/julien-blanchard/dbs/main/lol.json"

def getData(url):
    response = urlopen(url)
    data = json.loads(response.read())
    return data

parsed = getData(lol_reduced)

In [7]:
# to loop through a dict / json file, you use .items(), which splits the data into keys and values

# simple example
for author,data in parsed.items():
    print(author,data)

Aatrox {'id': 266, 'key': 'Aatrox', 'name': 'Aatrox', 'title': 'the Darkin Blade', 'fullName': '', 'icon': 'http://ddragon.leagueoflegends.com/cdn/12.19.1/img/champion/Aatrox.png', 'resource': 'BLOOD_WELL', 'attackType': 'MELEE', 'adaptiveType': 'PHYSICAL_DAMAGE', 'stats': {'health': {'flat': 650, 'percent': 0.0, 'perLevel': 114, 'percentPerLevel': 0.0}, 'healthRegen': {'flat': 3, 'percent': 0.0, 'perLevel': 1, 'percentPerLevel': 0.0}, 'mana': {'flat': 0, 'percent': 0.0, 'perLevel': 0, 'percentPerLevel': 0.0}, 'manaRegen': {'flat': 0, 'percent': 0.0, 'perLevel': 0, 'percentPerLevel': 0.0}, 'armor': {'flat': 38, 'percent': 0.0, 'perLevel': 4.45, 'percentPerLevel': 0.0}, 'magicResistance': {'flat': 32, 'percent': 0.0, 'perLevel': 2.05, 'percentPerLevel': 0.0}, 'attackDamage': {'flat': 60, 'percent': 0.0, 'perLevel': 5, 'percentPerLevel': 0.0}, 'movespeed': {'flat': 345, 'percent': 0.0, 'perLevel': 0.0, 'percentPerLevel': 0.0}, 'acquisitionRadius': {'flat': 475, 'percent': 0.0, 'perLevel'

In [19]:
# so, we now to dive into the nested values
for author,data in parsed.items():
    for k,v in data.items():
        print(f"{author:<8}=>\t{k:<20}=>\t{v}")

"""
the problem, as you can see, is that there are even more sub-nested data within your json file.
This means that for each subnest, you'll have again to loop through the nested data struct, etc..
ut to be fair Nate, I don't think you'll need ALL that data on your table
What you can do instead, is either pick which columns your want, OR create multiple Pandas dataframes and save them
Onto different SQL tables
"""

Aatrox  =>	id                  =>	266
Aatrox  =>	key                 =>	Aatrox
Aatrox  =>	name                =>	Aatrox
Aatrox  =>	title               =>	the Darkin Blade
Aatrox  =>	fullName            =>	
Aatrox  =>	icon                =>	http://ddragon.leagueoflegends.com/cdn/12.19.1/img/champion/Aatrox.png
Aatrox  =>	resource            =>	BLOOD_WELL
Aatrox  =>	attackType          =>	MELEE
Aatrox  =>	adaptiveType        =>	PHYSICAL_DAMAGE
Aatrox  =>	stats               =>	{'health': {'flat': 650, 'percent': 0.0, 'perLevel': 114, 'percentPerLevel': 0.0}, 'healthRegen': {'flat': 3, 'percent': 0.0, 'perLevel': 1, 'percentPerLevel': 0.0}, 'mana': {'flat': 0, 'percent': 0.0, 'perLevel': 0, 'percentPerLevel': 0.0}, 'manaRegen': {'flat': 0, 'percent': 0.0, 'perLevel': 0, 'percentPerLevel': 0.0}, 'armor': {'flat': 38, 'percent': 0.0, 'perLevel': 4.45, 'percentPerLevel': 0.0}, 'magicResistance': {'flat': 32, 'percent': 0.0, 'perLevel': 2.05, 'percentPerLevel': 0.0}, 'attackDamage': {'flat': 

In [36]:
"""
Now, imagine that you want to create a dataframe for the first few key / value pairs
You don't have to do it this way, but I would create a C like "struct" first, for readability
"""

def getStruct():
    struct = {
        "id": [],
        "key": [],
        "name": [],
        "resource": [],
        "attackType": [],
        "adaptiveType": []
    }
    return struct

# now we loop through the first level of key / value pairs and append the values to the struct containers
def getData(data):
    s = getStruct()
    for author,data in data.items():
        s["id"].append(data["id"])
        s["key"].append(data["key"])
        s["name"].append(data["name"])
        s["resource"].append(data["resource"])
        s["attackType"].append(data["attackType"])
        s["adaptiveType"].append(data["adaptiveType"])
    return s

# now you can create a Pandas dataframe! Funnily enough, Pandas accepts dictionaries, aka simple level JSON files
def getDataFrame(data):
    s = getData(data)
    dframe = pd.DataFrame(s)
    return dframe

df = getDataFrame(parsed)

df.head()

,id,key,name,resource,attackType,adaptiveType
0,266,Aatrox,Aatrox,BLOOD_WELL,MELEE,PHYSICAL_DAMAGE
1,103,Ahri,Ahri,MANA,RANGED,MAGIC_DAMAGE


# **SQL Alchemy**

In [37]:
import sqlalchemy
from sqlalchemy import create_engine

In [38]:
def getDataBase(db_name_in,db_name_out,dataframe):
  engine = create_engine(f"sqlite:///{db_name_in}.db") # Using SQLite
  #engine = create_engine(f"mysql:///{db_name_in}.db") # that would be for MySQL
  dataframe.to_sql(db_name_out, engine, index=False, if_exists="replace") # instead of "replace" you can put "append" for updates
  return engine

# we're creating a database called lol, and storing the dataframe into a table called author
# now, you could create multiple tables within your database, like say "elements", and join then on the author ID
engine = getDataBase("lol","author",df)

In [39]:
# making sure it worked
df = pd.read_sql("author", engine)
df

,id,key,name,resource,attackType,adaptiveType
0,266,Aatrox,Aatrox,BLOOD_WELL,MELEE,PHYSICAL_DAMAGE
1,103,Ahri,Ahri,MANA,RANGED,MAGIC_DAMAGE
